In [1]:
# import pandas as pd

# # Load the dataset
# data = pd.read_csv("/Users/john-zaatri/Desktop/semester 7/פרויקט מסכם/tabular data/DATA for AI.csv", low_memory=False)

# # Define the list of columns to keep
# columns_to_keep = ['Ocular Treatment ID', 'File', 'Age', 'Gender', 'Eye', 'Dominant Eye',
#                    'Pachymetry', 'PRE-OP Average K', 'PRE-OP K Axis max',
#                    'Pre-op K  Axis min', 'Contact Lens', 'Free of CL', 'Free of CL Unit',
#                    'UCVA', 'Subjective SEQ', 'Subjective Sph', 'Subjective Cyl', 'Subjective Cyl Axis', 
#                    'Subjective BCVA', 'Cycloplegia Sph', 'Cycloplegia Cyl', 'Cycloplegia Axis',
#                    'Cycloplegia CVA', 'Surgery Date', 'Treatment SEQ', 'Treatment Param Axis', 
#                    'Opt Zo', 'Max Abl Depth', 'Treatment Type', 'Humidity', 'Temp', 'Op.Time', 
#                    'Micro', 'Ring', 'Stop', 'Head', 'Alchohol', 'PTK mmm', 'MZ sec', 
#                    'Therapeutic Cont L', 'Rotation Angle', 'Rotation Angle Degrees', 
#                    'Last_Target Sph', 'Last_UCVA', 
                   
#                    'Last_Pachymetry', 'Last_Average K', 'Last_Steep Axis min', 'LASTSubjective SEQ', 'Last_Subjective Axis', 'Last_Subjective BCVA']

# # Filter the DataFrame to keep only the specified columns
# filtered_data = data.loc[:, columns_to_keep]

# # Define the file path for the new CSV file
# output_file_path = "/Users/john-zaatri/Desktop/semester 7/פרויקט מסכם/tabular data/Cleared-Data-Modified.csv"

# # Save the filtered data to a new CSV file
# filtered_data.to_csv(output_file_path, index=False)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
# Load the dataset
data = pd.read_csv("/Users/john-zaatri/Desktop/semester 7/פרויקט מסכם/tabular data/Cleared-Data-Modified.csv", low_memory=False)

In [4]:
print(data.columns)

Index(['Ocular Treatment ID', 'File', 'Age', 'Gender', 'Eye', 'Dominant Eye',
       'Pachymetry', 'PRE-OP Average K', 'PRE-OP K Axis max',
       'Pre-op K  Axis min', 'Contact Lens', 'Free of CL', 'Free of CL Unit',
       'UCVA', 'Subjective SEQ', 'Subjective Sph', 'Subjective Cyl',
       'Subjective Cyl Axis', 'Subjective BCVA', 'Cycloplegia Sph',
       'Cycloplegia Cyl', 'Cycloplegia Axis', 'Cycloplegia CVA',
       'Surgery Date', 'Treatment SEQ', 'Treatment Param Axis', 'Opt Zo',
       'Max Abl Depth', 'Treatment Type', 'Humidity', 'Temp', 'Op.Time',
       'Micro', 'Ring', 'Stop', 'Head', 'Alchohol', 'PTK mmm', 'MZ sec',
       'Therapeutic Cont L', 'Rotation Angle', 'Rotation Angle Degrees',
       'Last_Target Sph', 'Last_UCVA', 'Last_Pachymetry', 'Last_Average K',
       'Last_Steep Axis min', 'LASTSubjective SEQ', 'Last_Subjective Axis',
       'Last_Subjective BCVA'],
      dtype='object')


In [9]:
# Checking count of null values 
columns_to_check = ['Ocular Treatment ID', 'File', 'Age', 'Gender', 'Eye', 'Dominant Eye',
                    'Pachymetry', 'PRE-OP Average K', 'PRE-OP K Axis max',
                    'Pre-op K  Axis min', 'Contact Lens', 'Free of CL', 'Free of CL Unit',
                    'UCVA', 'Subjective SEQ', 'Subjective Sph', 'Subjective Cyl', 'Subjective Cyl Axis', 'Subjective BCVA',
                    'Cycloplegia Sph', 'Cycloplegia Cyl', 'Cycloplegia Axis',
                    'Cycloplegia CVA', 'Surgery Date', 'Treatment SEQ',
                    'Treatment Param Axis', 'Opt Zo', 'Max Abl Depth', 'Treatment Type',
                    'Humidity', 'Temp', 'Op.Time']

# Count null values for each column
null_counts = data[columns_to_check].isnull().sum()

print("Null value counts for each specified column:")
print(null_counts)

Null value counts for each specified column:
Ocular Treatment ID          0
File                         0
Age                        411
Gender                    1978
Eye                          0
Dominant Eye             33183
Pachymetry                3809
PRE-OP Average K          9130
PRE-OP K Axis max        17611
Pre-op K  Axis min       17368
Contact Lens             73367
Free of CL               64410
Free of CL Unit          64888
UCVA                      7876
Subjective SEQ             697
Subjective Sph             737
Subjective Cyl            4375
Subjective Cyl Axis       3679
Subjective BCVA            876
Cycloplegia Sph         112880
Cycloplegia Cyl         114616
Cycloplegia Axis        112873
Cycloplegia CVA         121006
Surgery Date                 0
Treatment SEQ              197
Treatment Param Axis     15024
Opt Zo                    1838
Max Abl Depth             1930
Treatment Type               0
Humidity                  4777
Temp                     

In [8]:
# Convert " " to None in the "Dominant Eye" column
data["Dominant Eye"] = data["Dominant Eye"].replace(" ", None)

Number of values with a space in the 'Dominant Eye' column: 0


In [10]:
# Handle data where 'Pre-op K Axis min' is NaN and 'PRE-OP K Axis max' is not NaN
def fill_pre_op_k_axis_max(df):
    # Identify the condition where 'Pre-op K Axis min' is NaN and 'PRE-OP K Axis max' is not NaN
    condition = df['Pre-op K  Axis min'].isnull() & df['PRE-OP K Axis max'].notnull()

    # Apply vectorized operations to adjust 'Pre-op K Axis min' based on 'PRE-OP K Axis max'
    df.loc[condition & (df['PRE-OP K Axis max'] > 90), 'Pre-op K  Axis min'] = df['PRE-OP K Axis max'] - 90
    df.loc[condition & (df['PRE-OP K Axis max'] <= 90), 'Pre-op K  Axis min'] = df['PRE-OP K Axis max'] + 90

    return df

df = fill_pre_op_k_axis_max(data)
df = df.drop('PRE-OP K Axis max',axis=1)

In [11]:
# This cell will handle the 3 columns of 'Contact Lens', 'Free of CL', 'Free of CL Unit' and merge them into one column while handling null values
def HoursToDays(df):
    # Vectorized operation to convert hours to days
    condition = (df['Free of CL Unit'] == "Hours") # | (df['Free of CL Unit'].notnull()) 
    df.loc[condition, 'Free of CL'] /= 24
    return df

def normalize_value(row, min_val, max_val):
    # Normalize 'Free of CL' value based on lens type
    val = row['Free of CL']
    if pd.isnull(val): #################################
        return 1 
    normalized = (val - min_val) / (max_val - min_val)
    return min(max(normalized, 0), 1)  # Ensure the normalized value is between 0 and 1

def mergeCL(df):
    df = HoursToDays(df)  # Convert hours to days first

    # Use 'apply' with a custom function for complex logic
    def apply_normalization(row):
        if row['Contact Lens'] == "Hard":
            return normalize_value(row, 7, 28)
        elif row['Contact Lens'] == "Soft":
            return normalize_value(row, 3, 7)
        else:
            return 1  # Default value for other types

    df['Merge_Contact Lens'] = df.apply(apply_normalization, axis=1)

    # Drop unnecessary columns
    df.drop(columns=['Contact Lens', 'Free of CL', 'Free of CL Unit'], inplace=True)

    return df

df = mergeCL(df)

In [12]:
# Update subjective SEQ values based on the columns Cycloplegia Sph and Cycloplegia Cyl
def update_subjective_seq(row):
    # Check if at least one value is not null between 'Cycloplegia Sph' and 'Cycloplegia Cyl'
    if pd.notnull(row['Cycloplegia Sph']) or pd.notnull(row['Cycloplegia Cyl']):
        # Check if 'Cycloplegia Sph' is not null and 'Cycloplegia Cyl' is not null
        if pd.notnull(row['Cycloplegia Sph']) and pd.notnull(row['Cycloplegia Cyl']):
            # Update 'Subjective SEQ' based on 'Cycloplegia Sph' and 'Cycloplegia Cyl'
            return row['Cycloplegia Sph'] + 0.5 * row['Cycloplegia Cyl']
        # Check if 'Cycloplegia Sph' is null and 'Cycloplegia Cyl' is not null
        elif pd.isnull(row['Cycloplegia Sph']) and pd.notnull(row['Cycloplegia Cyl']):
            # Update 'Subjective SEQ' based on 'Subjective Sph' and 'Cycloplegia Cyl'
            return row['Subjective Sph'] + 0.5 * row['Cycloplegia Cyl']
        # Check if 'Cycloplegia Cyl' is null and 'Cycloplegia Sph' is not null
        elif pd.isnull(row['Cycloplegia Cyl']) and pd.notnull(row['Cycloplegia Sph']):
            # Update 'Subjective SEQ' based on 'Cycloplegia Sph' and 'Subjective Cyl'
            return row['Cycloplegia Sph'] + 0.5 * row['Subjective Cyl']
    else:
        # If neither 'Cycloplegia Sph' nor 'Cycloplegia Cyl' is present, return the existing 'Subjective SEQ' value
        return row['Subjective SEQ']

# Assuming 'df' is your DataFrame containing the relevant columns
df['Subjective SEQ'] = df.apply(update_subjective_seq, axis=1)

# Drop the specified columns
df.drop(columns=['Cycloplegia Sph', 'Cycloplegia Cyl', 'Subjective Sph', 'Subjective Cyl'], inplace=True)

In [13]:
 # Update Subjective axis and CVA columns with Cycloplegia axis and cva values where they are not null
def MergeSubCycloplegia(df):
    cyclo_columns = ['Cycloplegia Axis', 'Cycloplegia CVA'] #################################
    subj_columns = ['Subjective Cyl Axis', 'Subjective BCVA'] #################################

    for cyclo, subj in zip(cyclo_columns, subj_columns):
        condition = df[cyclo].notnull()
        df.loc[condition, subj] = df.loc[condition, cyclo]

    # Drop the original 'Cycloplegia' columns
    df.drop(columns=cyclo_columns, inplace=True)

    return df

df = MergeSubCycloplegia(df)

In [14]:
# Print the counts for both conditions
print("Number of rows with null values in Rotation Angle after nulling values:", 
      ((df['Rotation Angle Degrees'].isnull()) & (df['Rotation Angle'].notnull())).sum())

print("Number of rows with 'CW' in Rotation Angle after filling values:", 
      ((df['Rotation Angle'].isnull()) & (df['Rotation Angle Degrees'].notnull())).sum())

# Null the values of Rotation Angle where Rotation Angle Degrees is null and Rotation Angle is not null
df.loc[df['Rotation Angle Degrees'].isnull() & df['Rotation Angle'].notnull(), 'Rotation Angle'] = pd.NA

# Fill Rotation Angle with "CW" where Rotation Angle is null and Rotation Angle Degrees is not null
df.loc[df['Rotation Angle'].isnull() & df['Rotation Angle Degrees'].notnull(), 'Rotation Angle'] = "CW"

Number of rows with null values in Rotation Angle after nulling values: 60
Number of rows with 'CW' in Rotation Angle after filling values: 6


In [15]:
# This cell merge the columns Rotation Angle and Rotation Angle Degrees (CW and CCW)
def LaserRotationMerge(df):
    # Identify rows where 'Rotation Angle' is 'CCW' and 'Rotation Angle Degrees' is not null
    condition = (df['Rotation Angle'] == 'CCW') & df['Rotation Angle Degrees'].notnull()

    # Multiply 'Rotation Angle Degrees' by -1 for rows that meet the condition
    df.loc[condition, 'Rotation Angle Degrees'] *= -1

    # Drop the 'Rotation Angle' column
    df.drop('Rotation Angle', axis=1, inplace=True)

    return df

df = LaserRotationMerge(df)

In [16]:
# Remove rows where last ucva (target label) is null and tratement type equal to zeiss premium
df = df[df['Last_UCVA'].notnull()]
df = df[df['Treatment Type'] != "Zeiss Premium"]

In [17]:
# Handling the surgery date column
# Define a function to map the season based on the month
def map_season(month):
    if month in [3, 4, 5]:  # Spring: March (3) to May (5)
        return 1
    elif month in [6, 7, 8]:  # Summer: June (6) to August (8)
        return 2
    elif month in [9, 10, 11]:  # Fall: September (9) to November (11)
        return 3
    else:  # Winter: December (12) to February (2)
        return 4
    
# Define a function to map the year range based on the era
def map_era(year):
    if year >= 2000 and year <= 2005:
        return 1
    elif year >= 2006 and year <= 2010:
        return 2
    elif year >= 2011 and year <= 2015:
        return 3
    elif year >= 2016 and year <= 2020:
        return 4
    elif year >= 2021 and year <= 2022:
        return 5
    else:
        return 0  # Default value if the year doesn't fall into any era
    
# Extract month and year from 'Surgery Date'
df['Surgery Month'] = pd.to_datetime(df['Surgery Date']).dt.month
df['Surgery Year'] = pd.to_datetime(df['Surgery Date']).dt.year

# Map season and year range
df['Season'] = df['Surgery Month'].apply(map_season)
df['Era'] = df['Surgery Year'].apply(map_era)

# Drop the 'Surgery Date', 'Surgery Month', and 'Surgery Year' columns
df.drop(columns=['Surgery Date', 'Surgery Month', 'Surgery Year'], inplace=True)

/var/folders/1t/bbl6pwb16n501zzycpfhlml00000gn/T/ipykernel_5665/304131575.py:29: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Surgery Month'] = pd.to_datetime(df['Surgery Date']).dt.month
/var/folders/1t/bbl6pwb16n501zzycpfhlml00000gn/T/ipykernel_5665/304131575.py:30: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Surgery Year'] = pd.to_datetime(df['Surgery Date']).dt.year


In [18]:
# columns = ['Micro', 'Ring', 'Stop', 'Head', 'Alchohol', 'PTK mmm', 'MZ sec', 'Therapeutic Cont L', 'Rotation Angle Degrees']


# for col in columns:
#     # Filter the DataFrame for non-null values in the current column
#     filtered_df = df[df[col].notnull()]
    
#     # Group the filtered DataFrame by 'Treatment Type' and count the rows for each group
#     grouped_counts = filtered_df.groupby('Treatment Type').size()
    
#     # Print the unique Treatment Types and their corresponding row counts
#     print(f"Unique Treatment Types and row counts for non-null values in '{col}':")
#     for treatment_type, count in grouped_counts.items():
#         print(f"- {treatment_type}: {count} rows")
#     print()  # Add a new line for readability

In [19]:
# # Count null values in each column that needs to be filled
# null_counts = df[['Pachymetry', 'PRE-OP Average K', 'Pre-op K  Axis min', 
#                   'Subjective SEQ', 'Subjective Cyl Axis', 'Subjective BCVA', 'UCVA']].isnull().sum()
# print(null_counts)
# # List of columns to fill null values
# columns_to_fill = ['Pachymetry', 'PRE-OP Average K', 'Pre-op K  Axis min', 
#                    'Subjective SEQ', 'Subjective Cyl Axis', 'Subjective BCVA', 'UCVA']

# # List of corresponding help columns
# help_columns = ['Last_Pachymetry', 'Last_Average K', 'Last_Steep Axis min', 
#                 'LASTSubjective SEQ', 'Last_Subjective Axis', 'Last_Subjective BCVA', 'Last_UCVA']

# # Iterate over each column to be filled and its corresponding help column
# for col_fill, col_help in zip(columns_to_fill, help_columns):
#     # Count rows where the column to fill has null values and the help column has non-null values
#     count = df[df[col_fill].isnull() & df[col_help].notnull()].shape[0]
#     print(f"Number of rows where {col_fill} is null and {col_help} is not null:", count)



In [20]:
differences_df = pd.DataFrame()

# List of before-surgery columns
before_surgery_columns = ['Pachymetry', 'PRE-OP Average K', 'Pre-op K  Axis min', 
                          'Subjective SEQ', 'Subjective Cyl Axis', 'Subjective BCVA', 'UCVA']

# List of after-surgery columns
after_surgery_columns = ['Last_Pachymetry', 'Last_Average K', 'Last_Steep Axis min', 
                         'LASTSubjective SEQ', 'Last_Subjective Axis', 'Last_Subjective BCVA', 'Last_UCVA']

# Iterate over pairs of before-surgery and after-surgery columns
for before_col, after_col in zip(before_surgery_columns, after_surgery_columns):
    # Calculate the difference between after-surgery and before-surgery values
    differences_df[after_col] = df[after_col] - df[before_col]

# Calculate the average difference for each after-surgery column
average_differences = differences_df.mean()

# Print the average differences and overall change
print("Average Differences:")
print(average_differences)

# Fill null values in before-surgery columns using calculated differences
for before_col, after_col in zip(before_surgery_columns, after_surgery_columns):
    # Identify rows where the after-surgery column is not null
    not_null_mask = df[after_col].notnull()

    # Calculate the imputed values using the average difference
    imputed_values = df[after_col][not_null_mask] - average_differences[after_col]

    # Update null values in the before-surgery column with imputed values
    df.loc[df[before_col].isnull() & not_null_mask, before_col] = imputed_values
    
# List of after-surgery columns to drop
after_surgery_columns_to_drop = [col for col in after_surgery_columns if col != 'Last_UCVA']

# Drop the after-surgery columns except 'Last_UCVA' from the DataFrame
df.drop(after_surgery_columns_to_drop, axis=1, inplace=True)

Average Differences:
Last_Pachymetry        -73.377499
Last_Average K          -3.396130
Last_Steep Axis min     -2.121525
LASTSubjective SEQ       3.480470
Last_Subjective Axis   -24.362817
Last_Subjective BCVA    -0.003787
Last_UCVA                0.781154
dtype: float64


In [21]:
# List of columns to fill with mean value
columns_to_fill_mean = ['Age', 'Pachymetry', 'PRE-OP Average K', 'Pre-op K  Axis min', 'Subjective SEQ',
                        'Subjective Cyl Axis', 'Subjective BCVA', 'Treatment SEQ', 'Treatment Param Axis',
                        'Opt Zo', 'Max Abl Depth', 'Humidity', 'Temp']

# List of columns to fill with mode value
columns_to_fill_mode = ['Gender', 'Dominant Eye']

# Fill numerical columns with mean value
df[columns_to_fill_mean] = df[columns_to_fill_mean].fillna(df[columns_to_fill_mean].mean())

# Fill categorical columns with mode value
df[columns_to_fill_mode] = df[columns_to_fill_mode].fillna(df[columns_to_fill_mode].mode().iloc[0])

In [22]:
# Define the neutral value for numerical columns
neutral_value_numeric = -2  # A numerical value that is outside the range of your actual data

# Fill the numerical columns with the neutral value
df['Ring'] = df['Ring'].fillna(neutral_value_numeric)
df['Stop'] = df['Stop'].fillna(neutral_value_numeric)

# For the textual columns, use the previous approach
neutral_value_textual = "Unknown"
textual_columns_to_fill_neutral = ['Micro', 'Head']

# Fill the specified textual columns with the neutral value
df[textual_columns_to_fill_neutral] = df[textual_columns_to_fill_neutral].fillna(neutral_value_textual)

In [23]:
neutral_value_numeric = -2  # Choose a numerical value that is outside the range of your actual data

# Fill the numerical columns with the neutral value
df['Alchohol'] = df['Alchohol'].fillna(neutral_value_numeric)
df['PTK mmm'] = df['PTK mmm'].fillna(neutral_value_numeric)
df['MZ sec'] = df['MZ sec'].fillna(neutral_value_numeric)

In [24]:
df['Therapeutic Cont L'] = df['Therapeutic Cont L'].fillna("No")

df['Rotation Angle Degrees'] = df['Rotation Angle Degrees'].fillna(0)

In [25]:
# Checking count of null values 
columns_to_check = ['Ocular Treatment ID', 'File', 'Age', 'Gender', 'Eye', 'Dominant Eye',
                    'Pachymetry', 'PRE-OP Average K',
                    'Pre-op K  Axis min', 'Merge_Contact Lens',
                    'UCVA', 'Subjective SEQ', 'Subjective Cyl Axis', 'Subjective BCVA', 'Era', 'Season', 'Treatment SEQ',
                    'Treatment Param Axis', 'Opt Zo', 'Max Abl Depth', 'Treatment Type',
                    'Humidity', 'Temp', 'Op.Time',
                    
                    'Micro', 'Ring', 'Stop', 'Head', 'Alchohol', 'PTK mmm', 'MZ sec', 
                    'Therapeutic Cont L', 'Rotation Angle Degrees', 
                    'Last_Target Sph', 'Last_UCVA']

# Count null values for each column
null_counts = df[columns_to_check].isnull().sum()

print("Null value counts for each specified column:")
print(null_counts)

Null value counts for each specified column:
Ocular Treatment ID       0
File                      0
Age                       0
Gender                    0
Eye                       0
Dominant Eye              0
Pachymetry                0
PRE-OP Average K          0
Pre-op K  Axis min        0
Merge_Contact Lens        0
UCVA                      0
Subjective SEQ            0
Subjective Cyl Axis       0
Subjective BCVA           0
Era                       0
Season                    0
Treatment SEQ             0
Treatment Param Axis      0
Opt Zo                    0
Max Abl Depth             0
Treatment Type            0
Humidity                  0
Temp                      0
Op.Time                   0
Micro                     0
Ring                      0
Stop                      0
Head                      0
Alchohol                  0
PTK mmm                   0
MZ sec                    0
Therapeutic Cont L        0
Rotation Angle Degrees    0
Last_Target Sph           0
Las

In [26]:
# Assuming your DataFrame is named df
# df.to_csv('/Users/john-zaatri/Desktop/semester 7/פרויקט מסכם/tabular data/processed_data.csv', index=False)